In [ ]:
from typing import Union
import numpy as np
from itertools import combinations
import time

**Добавим декоратор времени**

In [ ]:
def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Время выполнения функции {func.__name__}: {end_time - start_time:.5f} секунд")
        return result

    return wrapper

**Создание порождающей-канонической матрицы**

Функция `is_linearly_independent` проверяет на то, что матрица линейно независима.

Функция `canonical_matrix` возвращает порождающую-каноническую матрицу.

In [ ]:
def is_linearly_independent(k: int, n: int, field: int) -> np.array:
    while True:
        matrix = np.random.randint(0, field, size=(k, n - k))
        if np.linalg.matrix_rank(matrix) == min(k, n - k):
            return matrix


def canonical_matrix(k: int, n: int, field: int) -> np.array:
    return np.hstack((np.identity(k, dtype=int), is_linearly_independent(k, n, field)))

**Создание проверочной матрицы**

Функция `check_matrix` возвращает проверочную матрицу, на вход подаётся матрица `g` и поле Галуа `field`.

In [ ]:
def check_matrix(g: np.array, field: int) -> np.array:
    k, n = g.shape
    return np.hstack((np.mod(-g[:, k:].T, field), np.identity(n - k, dtype=int)), dtype=int)

**Создание канонических-порождающих матриц, создание проверочных для них матриц**

 Проверка на правильность работы показана тем, что `g @ h.T` равняется нулевой матрице.

In [ ]:
g1 = canonical_matrix(2, 4, 2)
g2 = canonical_matrix(11, 15, 2)
g3 = canonical_matrix(4, 6, 3)
g4 = canonical_matrix(4, 8, 3)

h1 = check_matrix(g1, 2)
h2 = check_matrix(g2, 2)
h3 = check_matrix(g3, 3)
h4 = check_matrix(g4, 3)

fields = [2, 2, 3, 3]
for i, field in enumerate(fields, start=1):
    g = globals()["g" + str(i)]
    h = globals()["h" + str(i)]
    gh = (g @ h.T) % field  # Calculate G * H.T modulo field
    print(f"Матрица g{i}:\n{np.array2string(g, separator=' ')}")
    print(f"Матрица h{i}:\n{np.array2string(h, separator=' ')}")
    print(f"Произведение g{i} * h{i}.T:\n{np.array2string(gh, separator=' ')}")
    print()

**Декодирование с помощью таблицы стандартного расположения**

В следующей ячейке написаны допольнительные функции для последующих заданий декодирования.

Функция `generate_new_error_vector` генерирует новый вектор ошибки, который не содержится в заданной таблице ошибок.

Функция `generate_table` создаёт таблицу стандартного расположения.

In [ ]:
def generate_new_error_vector(table, field: int):
    n = table[0].shape[1]
    table_set = set(map(tuple, np.unique(np.vstack(table), axis=0)))
    for weight in range(1, n + 1):
        for vector in combinations(range(n), weight):
            new_vector = np.zeros(n, dtype=int)
            new_vector[list(vector)] = 1
            new_vector = np.mod(new_vector, field)
            if tuple(new_vector) not in table_set:
                return new_vector
    return None


def generate_table(g: np.array, field: int) -> np.array:
    k, n = g.shape
    all_vectors = np.stack(np.meshgrid(*[range(field) for _ in range(k)])).T.reshape(-1, k)
    syndrome_vectors = np.mod(all_vectors @ g, field)
    line = syndrome_vectors.reshape(field ** k, n)
    table = np.array([line])
    error_vector = generate_new_error_vector(table, field)
    while error_vector is not None:
        table = np.append(table, [np.mod(error_vector + line, field)], axis=0)
        error_vector = generate_new_error_vector(table, field)
    return table

**Создание таблиц**

In [ ]:
%%time
table_1 = generate_table(g1, 2)
table_2 = generate_table(g2, 2)
table_3 = generate_table(g3, 3)
table_4 = generate_table(g4, 3)

**Декодирование с помощью таблицы стандартного расположения**

Функция `decode_standard_table` декодирует с помощью таблицы стандартного расположения.

In [ ]:
@timer
def decode_standard_table(table, vector: np.array, field: int) -> Union[np.array, None]:
    matches = np.where(np.all(table == vector[np.newaxis, :], axis=2))
    if len(matches[0]) == 0:
        return None
    i, j = matches[0][0], matches[1][0]
    result = np.mod(vector - table[i][0], field)
    return result

**Декодирование по синдрому**

Функция `decode_syndrome` деодирует по синдрому. 

In [ ]:
@timer
def decode_syndrome(table: np.array, vector: np.array, g: np.array, field) -> Union[str, np.array]:
    h = check_matrix(g, field)
    s = np.mod(vector @ h.T, field)
    if np.count_nonzero(s) == 0:
        return vector

    syndrome_products = np.mod(table[:, 0] @ h.T, field)
    matched_indices = np.where(np.all(s == syndrome_products, axis=-1))[0]

    if matched_indices.size == 0:
        return 'Отказ от декодирования'

    matched_line = table[matched_indices[0]]
    weight_line = np.count_nonzero(matched_line, axis=1)
    min_weight = weight_line[0]
    e = matched_line[0]

    if np.any(weight_line[1:] <= min_weight):
        return 'Отказ от декодирования'

    return np.mod(vector - e, field)

**Функция зашумления канала**

Написал дополнительную функцию зашумления канала `make_noise`.

In [ ]:
def make_noise(p, field, bit):
    alphabet = set(i for i in range(field))
    if np.random.random() < p:
        # выбираем новый бит, отличный от исходного
        new_bit = np.random.choice([b for b in alphabet if b != bit])
    else:
        # если бит не был изменен, возвращаем исходное значение
        new_bit = bit
    return new_bit, int(new_bit != bit)

**ТЕСТЫ**

Функция `for_tests` выводит результаты декодирования с помощью функций `decode_standard_table` и `decode_standard_table`. 

Функция `tests` запускает тесты, задаёт их количество {чтобы изменить количество тестов, можно поменять цифру в цикле for} и создаёт рандомные сообщения для передачи.

In [ ]:
def for_tests(table: np.array, vector: np.array, g: np.array, field: int, p=0.3, fine=0, bad=0, refusal=0):
    vector_len = g.shape[0]
    encoded = np.mod(vector @ g, field)
    noised, errors = np.vectorize(make_noise)(p, field, encoded)
    decoded_ST = decode_standard_table(table, noised, field)[:vector_len]
    decoded_SYN = decode_syndrome(table, noised, g, field)
    if type(decoded_SYN) != type(''):
        decoded_SYN = decoded_SYN[:vector_len]
        refusal += 1
    print('Исходное сообщение:      ', vector)
    print('Закодированное сообщение:', encoded)
    print('Зашумлённое сообщение:   ', noised)
    print('Через таблицу:           ', decoded_ST)
    print('По синдрому:             ', decoded_SYN)
    if np.array_equal(vector, decoded_ST):
        print(f'Декодирование прошло успешно, ошибок: {np.count_nonzero(errors)}')
        fine += 1
    else:
        print(f'Декодирование не успешно, ошибок: {np.count_nonzero(errors)}')
        bad += 1
    print()
    return fine, bad, refusal


def tests(table: np.array, g: np.array, field: int):
    k = g.shape[0]
    fine = 0
    bad = 0
    refusal=0
    for _ in range(20):
        vector = np.random.randint(0, field ** k)
        vector = np.array([int(digit) for digit in np.base_repr(vector, base=field).zfill(k)])
        fine, bad, refusal = for_tests(table, vector, g, field, fine=fine, bad=bad, refusal=refusal)
        for_tests(table, vector, g, field)
    print(f'Успешных декодирований: {fine}, неуспешных декодирований: {bad}')
    print(f'Процент успешных декодированиий {100 * fine / (fine + bad)}%')
    print(f'Процент не успешных декодированиий {100 * bad / (fine + bad)}%')
    print(f'Отказов от декодирования {(20 - refusal) / 0.2}%')

**Тесты для первой матрицы**

In [ ]:
tests(table_1, g1, 2)

**Тесты для второй матрицы**

In [ ]:
tests(table_2, g2, 2)

**Тесты для третьей матрицы**

In [ ]:
tests(table_3, g3, 3)

**Тесты для четвёртой матрицы**

In [ ]:
tests(table_4, g4, 3)

**Ответы на вопросы**

1. В целом декодирование по синдрому и декодирование по стандратному расположению отличаются только на больших таблицах примеро на 0.001 секунды. Чтобы изменить количество тестов надо немного поменять код. В целом декодирование из очень оптимизированного алгоритма работает почти мнгновенно для небольших матриц.

2. Я провёл тестирование для 10.000 случайных векторов.  
   Для `table_1`:  
   Успешных декодирований: `5459`, неуспешных декодирований: `4541`  
   Процент успешных декодированиий `54.59%`  
   Процент не успешных декодированиий `45.41%`  
   Отказов от декодирования `26.38%`  
   Исправляет `1` ошибку примерно в половине случаев. 
         
   Для `table_2`:  
   Успешных декодирований: `307`, неуспешных декодирований: `9693`  
   Процент успешных декодированиий `3.07%`  
   Процент не успешных декодированиий `96.93% ` 
   Отказов от декодирования `55.3%`  
   Может исправлять максимум `1` ошибку.    
         
   Для `table_3`:  
   Успешных декодирований: 2684, неуспешных декодирований: 7316  
   Процент успешных декодированиий 26.84%  
   Процент не успешных декодированиий 73.16%  
   Отказов от декодирования 82.04%   
   Исправляет не больше `1` ошибки
          
   Для `table_4`:  
   Успешных декодирований: 2090, неуспешных декодирований: 7910  
   Процент успешных декодированиий 20.9%  
   Процент не успешных декодированиий 79.1%  
   Отказов от декодирования 83.12%  
   Исправляет до `3` ошибок.
